# Poles, Zeros, Z-Transform Structure, and FIR vs IIR Filters
**Continuation of the transforms notebook**  

Topics:
- Poles and zeros in the z-plane  
- How they arise from the Z-transform  
- Frequency response from poles/zeros  
- FIR vs IIR filters  
- Infinite impulse responses and recurrence relations  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_zplane(poles=None, zeros=None, title="z-plane"):
    theta = np.linspace(0, 2*np.pi, 400)
    unit_x = np.cos(theta)
    unit_y = np.sin(theta)

    fig, ax = plt.subplots(figsize=(5,5))
    ax.axhline(0, linewidth=1)
    ax.axvline(0, linewidth=1)
    ax.plot(unit_x, unit_y)
    
    if zeros is not None and len(zeros) > 0:
        ax.plot(np.real(zeros), np.imag(zeros), 'o', label="zeros")
    if poles is not None and len(poles) > 0:
        ax.plot(np.real(poles), np.imag(poles), 'x', markersize=10, label="poles")

    ax.set_aspect('equal', 'box')
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_title(title)
    ax.set_xlabel("Re{z}")
    ax.set_ylabel("Im{z}")
    ax.legend()
    plt.show()


## 1) Rational Z-transforms and transfer functions

For an LTI discrete-time system, the Z-transform of the impulse response \(h[n]\) is:

$$H(z) = \sum_{n=0}^{\infty} h[n] z^{-n}$$

In most practical systems, this becomes a **rational function**:

$$H(z) = \frac{B(z)}{A(z)} = \frac{b_0 + b_1 z^{-1} + \cdots + b_M z^{-M}}
{1 + a_1 z^{-1} + \cdots + a_N z^{-N}}$$

- **Zeros**: roots of \(B(z)=0\)  
- **Poles**: roots of \(A(z)=0\)


## 2) Poles and zeros in the z-plane

Write the transfer function in factored form:

$$H(z) = K \frac{\prod_{k=1}^{M} (1 - z_k^{-1} z)}
{\prod_{m=1}^{N} (1 - p_m^{-1} z)}
$$

- \(z_k\) = zeros  
- \(p_m\) = poles  

Each zero introduces a **notch** in the frequency response near its angle.  
Each pole introduces a **peak / resonance** near its angle.


### Example: simple pole and zero

$$
H(z) = \frac{1 - 0.8 z^{-1}}{1 - 0.5 z^{-1}}
$$

- Zero at \(z=0.8\)  
- Pole at \(z=0.5\)  


In [ ]:
# Plot pole-zero diagram
zeros = np.array([0.8])
poles = np.array([0.5])

plot_zplane(poles=poles, zeros=zeros, title="Pole-zero plot: one zero, one pole")

# Frequency response
w = np.linspace(0, np.pi, 512)
z = np.exp(1j*w)
H = (1 - 0.8*z**-1) / (1 - 0.5*z**-1)

fig, ax = plt.subplots(figsize=(7,4))
ax.plot(w, np.abs(H))
ax.set_title("Magnitude response |H(e^{jω})|")
ax.set_xlabel("ω (rad/sample)")
ax.set_ylabel("|H|")
plt.show()


## 3) FIR filters

A FIR filter:

$$H(z) = b_0 + b_1 z^{-1} + \cdots + b_M z^{-M}$$

- Only zeros, no poles  
- Always stable  
- Impulse response is finite  


In [ ]:
# Simple FIR moving average
b = np.ones(3)/3

w = np.linspace(0, np.pi, 512)
z = np.exp(1j*w)
H = b[0] + b[1]*z**-1 + b[2]*z**-2

fig, ax = plt.subplots(figsize=(7,4))
ax.plot(w, np.abs(H))
ax.set_title("FIR moving average magnitude response")
ax.set_xlabel("ω")
ax.set_ylabel("|H|")
plt.show()

zeros = np.roots(b[::-1])
plot_zplane(poles=[], zeros=zeros, title="FIR: only zeros")


## 4) IIR filters and recurrence relations

General IIR difference equation:

$$
y[n] = -\sum_{k=1}^{N} a_k y[n-k] + \sum_{m=0}^{M} b_m x[n-m]
$$

This feedback creates poles and an infinite impulse response.


In [ ]:
# First-order IIR example
N = 50
h = np.zeros(N)
x = np.zeros(N)
x[0] = 1.0

for n in range(N):
    h[n] = x[n]
    if n-1 >= 0:
        h[n] += 0.9 * h[n-1]

fig, ax = plt.subplots(figsize=(7,4))
ax.stem(np.arange(N), h, basefmt=" ")
ax.set_title("Impulse response of first-order IIR")
ax.set_xlabel("n")
ax.set_ylabel("h[n]")
plt.show()

plot_zplane(poles=[0.9], zeros=[], title="First-order IIR: one pole at 0.9")
